In [59]:
!pip install cufflinks

In [60]:
import pandas as pd
import cufflinks as cf
from collections import Counter

In [61]:
raw_df = pd.read_csv('#file path')
raw_df

File_path                                           Articles  \
0     politics  Budget to set scene for election..Gordon Brown...   
1     politics  Army chiefs in regiments decision..Military ch...   
2     politics  Howard denies split over ID cards..Michael How...   
3     politics  Observers to monitor UK election..Ministers wi...   
4     politics  Kilroy names election seat target..Ex-chat sho...   
...        ...                                                ...   
2219  business  India opens skies to competition..India will a...   
2220  business  Yukos bankruptcy 'not US matter'..Russian auth...   
2221  business  Survey confirms property slowdown..Government ...   
2222  business  High fuel prices hit BA's profits..British Air...   
2223  business  US trade gap hits record in 2004..The gap betw...   

                                              Summaries  
0     - Increase in the stamp duty threshold from £6...  
1     "They are very much not for the good and will ...  
2     Michael Howard has denied his shadow cabinet w...  
3     The report said individual registration should...  
4     UKIP's leader, Roger Knapman, has said he is g...  
...                                                 ...  
2219  India will allow domestic commercial airlines ...  
2220  Yukos says a US court was entitled to declare ...  
2221  House prices were 11.8% higher on the year in ...  
2222  Rod Eddington, BA's chief executive, said the ...  
2223  The Commerce Department said the trade deficit...  

[2224 rows x 3 columns]

In [62]:
ct = Counter(raw_df['File_path'])

In [63]:

pip install matplotlib


Note: you may need to restart the kernel to use updated packages.


# Distribution of Number of Articles in Each Category (visual represenation)


In [64]:
import matplotlib.pyplot as plt
import plotly.express as px

# Create a DataFrame from the counter
df = pd.DataFrame({"category":ct.keys(), "value":ct.values()})

In [65]:
# Create a bar chart
fig = px.bar(df, x="category", y="value")
# Show the chart
fig.show()

# Distribution Size of Each Category

In [66]:
# Create a bubble chart
fig = px.scatter(df, x="category", y="value", size="value", color="category")

# Show the chart
fig.show()

# Sentence Tokenization (text preprocessing)

In [68]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import numpy as np
import networkx as nx
import re

In [69]:
def read_article(text):
    sentences =[]
    sentences = sent_tokenize(text)
    for sentence in sentences:
        sentence.replace("[^a-zA-Z0-9]"," ")
    return sentences

In [70]:
article = raw_df["Articles"].iloc[5]
actual_summary = raw_df["Summaries"].iloc[5]

In [71]:
sent_tok = read_article(article)
sent_tok

['Donor attacks Blair-Brown \'feud\'..The reported feud between Tony Blair and Gordon Brown has prompted a Labour donor to say he will almost certainly refuse to give more funds...Duncan Bannatyne also attacked the government over Iraq and its "poor" response to the Asian tsunami crisis.',
 'His broadside came as ex-Foreign Secretary Robin Cook said he hoped Mr Brown would be premier at some point.',
 'Mr Bannatyne has previously given Labour £50,000.',
 'He made his fortune from care homes and health clubs...The 52-year-old on Tuesday said was he was reviewing his donations because of Cabinet disunity and international issues.',
 "His spokesman said it was highly unlikely he would give Labour more money, although he would remain a supporter and not fund the Conservatives...Robert Peston's new book has prompted more speculation about the Blair-Brown rift with its claims that the prime minister broke a promise made in 2003 to stand down.",
 'Mr Bannatyne said: "Disunity in the Cabinet h

# Spell Correction

In [72]:
pip install textblob


Note: you may need to restart the kernel to use updated packages.


In [73]:
pip install pyspellchecker


Note: you may need to restart the kernel to use updated packages.


In [74]:
from spellchecker import SpellChecker
import nltk
nltk.download("words")

def correct_sentence(sentence):
    spell = SpellChecker()
    corrected_tokens = []
    for token in sentence.split():
        corrected_token = spell.correction(token)
        if corrected_token is not None:  # Check if the correction is not None
            corrected_tokens.append(corrected_token)
        else:
            corrected_tokens.append(token)  # If correction is None, keep the original token
    corrected_sentence = " ".join(corrected_tokens)
    return corrected_sentence.capitalize()

mod_sent = []
for tok in sent_tok:  # Assuming `sent_tok` is already defined
    corrected_sent = correct_sentence(tok)
    print(f"\033[94m Original Token: {tok} \033[0m")  # Use correct color code
    print(f"\033[92m Corrected Token: {corrected_sent} \033[0m")  # Use correct color code
    mod_sent.append(corrected_sent)


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\DANICA\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


 Original Token: Donor attacks Blair-Brown 'feud'..The reported feud between Tony Blair and Gordon Brown has prompted a Labour donor to say he will almost certainly refuse to give more funds...Duncan Bannatyne also attacked the government over Iraq and its "poor" response to the Asian tsunami crisis. 
 Corrected Token: Donor attacks blair-brown 'feud'..the reported feud between tony flair and cordon brown has prompted a labor donor to say he will almost certainly refuse to give more funds...duncan bannatyne also attacked the government over iraq and its poor response to the avian tsunami crisis 
 Original Token: His broadside came as ex-Foreign Secretary Robin Cook said he hoped Mr Brown would be premier at some point. 
 Corrected Token: His broadside came as ex-foreign secretary robin cook said he hoped my brown would be premier at some point 
 Original Token: Mr Bannatyne has previously given Labour £50,000. 
 Corrected Token: My bannatyne has previously given labor £50,000. 
 Origin

In [75]:
" ".join(mod_sent)

"Donor attacks blair-brown 'feud'..the reported feud between tony flair and cordon brown has prompted a labor donor to say he will almost certainly refuse to give more funds...duncan bannatyne also attacked the government over iraq and its poor response to the avian tsunami crisis His broadside came as ex-foreign secretary robin cook said he hoped my brown would be premier at some point My bannatyne has previously given labor £50,000. He made his fortune from care homes and health clubs...the 52-year-old on tuesday said was he was reviewing his donations because of cabinet disunity and international issues His spokesman said it was highly unlikely he would give labor more money although he would remain a supporter and not fund the conservatives...robert piston's new book has prompted more speculation about the blair-brown rift with its claims that the prime minister broke a promise made in 2003 to stand down My bannatyne said disunity in the cabinet has a corrosive effect on the countr

# Sentence Similarity

In [76]:
pip install tensorflow_hub


Note: you may need to restart the kernel to use updated packages.


In [77]:
pip install --upgrade numpy


Note: you may need to restart the kernel to use updated packages.


In [78]:
pip install --upgrade tensorflow


Note: you may need to restart the kernel to use updated packages.


In [79]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

def sentence_similarity(sent1,sent2,embed):
    A = embed([sent1])[0]
    B = embed([sent2])[0]
    return 1 - (np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B)))


In [80]:
print(f"Number of elements in mod_sent: {len(mod_sent)}")


Number of elements in mod_sent: 17


In [81]:
import tensorflow_hub as hub
import numpy as np

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

def sentence_similarity(sent1, sent2, embed_model):
    embedding1 = embed_model([sent1])[0]
    embedding2 = embed_model([sent2])[0]
    similarity = np.inner(embedding1, embedding2)
    return similarity

In [82]:
def build_similarity_matrix(sentences,embeds):
    similarity_matrix = np.zeros((len(sentences),len(sentences)))
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1!=idx2:
                similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1],sentences[idx2],embeds)
    return similarity_matrix

In [83]:
sim_mat = build_similarity_matrix(mod_sent, embed)

In [84]:
print(f"\033[92m Sentence 1 : {mod_sent[0]}")
print(f"\033[92m Sentence 2 : {mod_sent[1]}")
print(f"\033[92m Similarity Score : {sentence_similarity(mod_sent[0], mod_sent[1], embed)}")

 Sentence 1 : Donor attacks blair-brown 'feud'..the reported feud between tony flair and cordon brown has prompted a labor donor to say he will almost certainly refuse to give more funds...duncan bannatyne also attacked the government over iraq and its poor response to the avian tsunami crisis
 Sentence 2 : His broadside came as ex-foreign secretary robin cook said he hoped my brown would be premier at some point
 Similarity Score : 0.32627952098846436


In [85]:
labeled_data = [
    {"sent1": "This is a sentence.", "sent2": "This is another sentence.", "similarity_score": 0.8},
    {"sent1": "A cat is sleeping.", "sent2": "The cat is awake.", "similarity_score": 0.6},
    # Add more labeled examples here
]

In [86]:
predicted_scores = []
true_scores = []

for example in labeled_data:
    sent1 = example["sent1"]
    sent2 = example["sent2"]
    similarity_score = example["similarity_score"]
    
    predicted_score = sentence_similarity(sent1, sent2, embed)
    
    predicted_scores.append(predicted_score)
    true_scores.append(similarity_score)

print("Predicted Scores:", predicted_scores)
print("True Scores:", true_scores)


    

Predicted Scores: [0.6809746, 0.7540814]
True Scores: [0.8, 0.6]


In [87]:
mse = np.mean((np.array(predicted_scores) - np.array(true_scores)) ** 2)
print("Mean Squared Error (MSE):", mse)


Mean Squared Error (MSE): 0.01895406583641411


In [88]:
pip install bokeh


Note: you may need to restart the kernel to use updated packages.


In [89]:
from bokeh.io import output_notebook, show
from bokeh.models import Range1d, Circle, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
import networkx as nx

output_notebook()

g = nx.Graph()

for i in range(sim_mat.shape[0]):
    for j in range(sim_mat.shape[1]):
        if sim_mat[i][j] >= .9:
            g.add_edge(i, j)

HOVER_TOOLTIPS = [("sent_tok", "@index")]
plot = figure(tooltips=HOVER_TOOLTIPS, tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
              x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1))

network_graph = from_networkx(g, nx.spring_layout, scale=7, center=(0, 0))
network_graph.node_renderer.glyph = Circle(radius=0.5, fill_color='green')
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)
plot.renderers.append(network_graph)
show(plot)


Loading BokehJS ...

In [90]:
def generate_summary(text,top_n,embeds):
    summarize_text = []
    sentences = read_article(text)
    sentence_similarity_matrix = build_similarity_matrix(sentences,embeds)
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)),reverse=True)
    for i in range(top_n):
        summarize_text.append(ranked_sentences[i][1])
    return " ".join(summarize_text)

In [91]:
def generate_summary(text, top_n=5, embeds=None):
    summarize_text = []
    sentences = nltk.sent_tokenize(text)
    if len(sentences) <= top_n:
        return text  # Return the original text if there are fewer sentences than top_n

    if embeds is None:
        embeds = embed_text(sentences)
    scores = score_sentences(sentences, embeds)

    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    
    # Ensure that there are enough sentences in ranked_sentences
    top_n = min(top_n, len(ranked_sentences))
    
    for i in range(top_n):
        summarize_text.append(ranked_sentences[i][1])
    return " ".join(summarize_text)


In [92]:
Original_Text = " ".join(mod_sent)
Summarized_Text = generate_summary(Original_Text, top_n=9, embeds=embed)

In [93]:
Original_Text

"Donor attacks blair-brown 'feud'..the reported feud between tony flair and cordon brown has prompted a labor donor to say he will almost certainly refuse to give more funds...duncan bannatyne also attacked the government over iraq and its poor response to the avian tsunami crisis His broadside came as ex-foreign secretary robin cook said he hoped my brown would be premier at some point My bannatyne has previously given labor £50,000. He made his fortune from care homes and health clubs...the 52-year-old on tuesday said was he was reviewing his donations because of cabinet disunity and international issues His spokesman said it was highly unlikely he would give labor more money although he would remain a supporter and not fund the conservatives...robert piston's new book has prompted more speculation about the blair-brown rift with its claims that the prime minister broke a promise made in 2003 to stand down My bannatyne said disunity in the cabinet has a corrosive effect on the countr

In [94]:
Summarized_Text

"Donor attacks blair-brown 'feud'..the reported feud between tony flair and cordon brown has prompted a labor donor to say he will almost certainly refuse to give more funds...duncan bannatyne also attacked the government over iraq and its poor response to the avian tsunami crisis His broadside came as ex-foreign secretary robin cook said he hoped my brown would be premier at some point My bannatyne has previously given labor £50,000. He made his fortune from care homes and health clubs...the 52-year-old on tuesday said was he was reviewing his donations because of cabinet disunity and international issues His spokesman said it was highly unlikely he would give labor more money although he would remain a supporter and not fund the conservatives...robert piston's new book has prompted more speculation about the blair-brown rift with its claims that the prime minister broke a promise made in 2003 to stand down My bannatyne said disunity in the cabinet has a corrosive effect on the countr

# BLEU SCORE AND ACCURACY

In [95]:
import nltk

hypothesis = Summarized_Text
reference = actual_summary
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
print(f"BLEUscore : {BLEUscore}")

BLEUscore : 0.36627177187824117


In [96]:
pip install nltk


Note: you may need to restart the kernel to use updated packages.


In [97]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [98]:
import nltk
from sklearn.metrics import accuracy_score

# Define the actual and predicted summaries
actual_summary = " ".join(mod_sent)
predicted_summary = Summarized_Text

# Convert the summaries to lists of words
actual_words = actual_summary.split()
predicted_words = predicted_summary.split()

# Calculate the accuracy
accuracy = accuracy_score(actual_words, predicted_words)

# Calculate the BLEU score
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)

# Print the accuracy and BLEU score
print(f"Accuracy: {accuracy}")
print(f"BLEUscore: {BLEUscore}")

Accuracy: 1.0
BLEUscore: 0.36627177187824117


In [99]:
# Define the actual and predicted summaries
actual_summary = " ".join(mod_sent)
predicted_summary = Summarized_Text

# Convert the summaries to lists of words
actual_words = actual_summary.split()
predicted_words = predicted_summary.split()

# Print the actual and predicted words
print("Actual Words:")
print(actual_words[:10])  # Print the first 10 words for inspection
print("\nPredicted Words:")
print(predicted_words[:10])  # Print the first 10 words for inspection


Actual Words:
['Donor', 'attacks', 'blair-brown', "'feud'..the", 'reported', 'feud', 'between', 'tony', 'flair', 'and']

Predicted Words:
['Donor', 'attacks', 'blair-brown', "'feud'..the", 'reported', 'feud', 'between', 'tony', 'flair', 'and']


In [101]:
import pandas as pd
from sklearn.metrics import confusion_matrix

# Define the actual and predicted summaries
actual_summary = " ".join(mod_sent)
predicted_summary = Summarized_Text

# Convert the summaries to lists of words
actual_words = actual_summary.split()
predicted_words = predicted_summary.split()

# Define the labels for the confusion matrix
labels = ["Dissimilar", "Similar"]

# Calculate the confusion matrix
try:
    conf_matrix = confusion_matrix(actual_words, predicted_words)
except ValueError as e:
    # Handle missing labels appropriately (e.g., by adjusting the labels or handling the missing labels)
    print("Error:", e)
    print("Attempting to calculate confusion matrix without labels...")
    conf_matrix = confusion_matrix(actual_words, predicted_words)

# Calculate TP, FP, TN, FN
TP = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
FN = conf_matrix[1, 0]
TN = conf_matrix[1, 1]

# Construct the 2x2 confusion matrix DataFrame
data = {
    "Predicted Dissimilar": [TP, FP],
    "Predicted Similar": [FN, TN]
}
index = ["Actual Dissimilar", "Actual Similar"]
df_conf_matrix = pd.DataFrame(data, index=index)

# Print the confusion matrix in tabular form
print("Confusion Matrix:")
print(df_conf_matrix)


Confusion Matrix:
                   Predicted Dissimilar  Predicted Similar
Actual Dissimilar                     1                  0
Actual Similar                        0                  2
